In [7]:
import pandas as pd

# Load the dataset
df = pd.read_excel(r"C:\Users\stuti\OneDrive\Desktop\temp_1\Census 2011 Gender and literacy rate wrt urban and rural divide.xlsx", engine="openpyxl")

# Filter for ages 7 and above
age_filter = ~df["Age-group"].isin(['0-6', 'All ages'])

# Group by State and Region, then calculate sums
grouped = df[age_filter].groupby(['Area Name', 'Regions'])
results = grouped.agg({
    'Total Males': 'sum',
    'Total Females': 'sum',
    'Literate Males': 'sum',
    'Literate Females': 'sum'
}).reset_index()

# Pivot the table to get Urban and Rural side by side
pivoted = results.pivot(index='Area Name', columns='Regions', 
                        values=['Total Males', 'Total Females', 'Literate Males', 'Literate Females'])

# Flatten column names
pivoted.columns = [f'{col[1]}_{col[0]}' for col in pivoted.columns]

# Reset index to make 'Area Name' a column again
pivoted.reset_index(inplace=True)

# Reorder columns for better readability
column_order = ['Area Name', 
                'Rural_Total Males', 'Rural_Total Females', 'Rural_Literate Males', 'Rural_Literate Females',
                'Urban_Total Males', 'Urban_Total Females', 'Urban_Literate Males', 'Urban_Literate Females']
pivoted = pivoted[column_order]

# Calculate total population for each category
pivoted['Rural_Total'] = pivoted['Rural_Total Males'] + pivoted['Rural_Total Females']
pivoted['Urban_Total'] = pivoted['Urban_Total Males'] + pivoted['Urban_Total Females']
pivoted['Rural_Total_Literate'] = pivoted['Rural_Literate Males'] + pivoted['Rural_Literate Females']
pivoted['Urban_Total_Literate'] = pivoted['Urban_Literate Males'] + pivoted['Urban_Literate Females']

# Rearrange columns to include totals
final_column_order = ['Area Name', 
                      'Rural_Total', 'Rural_Total Males', 'Rural_Total Females', 'Rural_Total_Literate', 'Rural_Literate Males', 'Rural_Literate Females',
                      'Urban_Total', 'Urban_Total Males', 'Urban_Total Females', 'Urban_Total_Literate', 'Urban_Literate Males', 'Urban_Literate Females']
pivoted = pivoted[final_column_order]

# Save to the same Excel file
output_path = r"C:\Users\stuti\OneDrive\Desktop\temp_1\Census 2011 Gender and literacy rate wrt urban and rural divide.xlsx"

# Read the existing Excel file
with pd.ExcelWriter(output_path, engine='openpyxl', mode='a') as writer:
    # Write the new data to a new sheet named 'State_wise_Summary_with_Literacy'
    pivoted.to_excel(writer, sheet_name='Gen_lit_rates', index=False)

print(f"Results have been saved to {output_path} in a new sheet named 'Gen_lit_rates'")

Results have been saved to C:\Users\stuti\OneDrive\Desktop\temp_1\Census 2011 Gender and literacy rate wrt urban and rural divide.xlsx in a new sheet named 'Gen_lit_rates'
